In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('/content/1. 실습용자료.txt', sep = '|', encoding = 'cp949')
test_df = pd.read_csv('/content/2. 모델개발용자료.txt', sep = '|', encoding = 'cp949')

In [ ]:
temp = train_df.digit_3.value_counts()[train_df.digit_3.value_counts() > 1000].index.to_list()
train_df = train_df.query('digit_3 == @temp')

In [ ]:
train_df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육
...,...,...,...,...,...,...,...
176273,id_0176274,I,56,561,음식점,접객시설을 갖추고,한식
176275,id_0176276,G,47,473,일반소비자에게,휴대폰매장에서,휴대폰
176276,id_0176277,I,56,562,영업장에서,접객시설을 갖추고,"커피,음료 판매"
176277,id_0176278,S,95,952,캬센타에서,특정 부분만을 전문적으로 수리,경정비.타이어 내장수리


In [ ]:
train_df= train_df.replace(np.nan, '', regex=True)
test_df= test_df.replace(np.nan, '', regex=True)

train_df['text'] = train_df['text_obj'] + " " + train_df['text_mthd'] + " " + train_df['text_deal']
test_df['text'] = test_df['text_obj'] + " " + test_df['text_mthd'] + " " + test_df['text_deal']

In [ ]:
train_df['digit_3'] = train_df['digit_3'].astype(str)

In [ ]:
import re 

punc = re.compile('[\!@#$%\^&\*\(\)\-\=\[\]\{\}\.,/\?~\+\'"|_:;><`┃]')

def remove_punctuation(text, lower = True):
    text = punc.sub(' ', str(text))
    text = ' '.join(text.split())
    if lower:
        text = text.lower()
    return text

train_df['text'] = train_df['text'].map(remove_punctuation)
test_df['text'] = test_df['text'].map(remove_punctuation)

In [ ]:
from sklearn.preprocessing import LabelEncoder

target_list = ['digit_3']


le = LabelEncoder()
train_df['digit_3'] = le.fit_transform(train_df['digit_3'])

train_df.head()

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
0,id_0000001,S,95,33,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차부분정비 타이어오일교환
1,id_0000002,G,47,10,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서 일반인을 대상으로 채소 과일판매
2,id_0000003,G,46,8,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여사업체에도매 공업용고무를가지고 합성고무도매
3,id_0000004,G,47,13,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반소비자에게 열쇠잠금장치
4,id_0000005,Q,87,29,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학전아동보육


In [ ]:
data_list = []
for q, label in zip(train_df['text'], train_df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
data_list[0]

['카센터에서 자동차부분정비 타이어오일교환', '33']

In [ ]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
!pip install mxnet

     |████████████████████████████████| 47.3 MB 1.5 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-hamoqz26
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-hamoqz26
     |████████████████████████████████| 132 kB 5.1 MB/s 
     |████████████████████████████████| 344 kB 39.6 MB/s 
     |████████████████████████████████| 4.5 MB 56.3 MB/s 
     |████████████████████████████████| 1.2 MB 44.2 MB/s 
     |████████████████████████████████| 4.0 MB 11.7 MB/s 
     |████████████████████████████████| 895 kB 46.5 MB/s 
     |████████████████████████████████| 6.5 MB 45.2 MB/s 
     |████████████████████████████████| 596 kB 49.9 MB/s 
     |████████████████████████████████| 77 kB 6.9 MB/s 
     |████████████████████████████████| 8.7 MB 38.1 MB/s 
     |████████████████████████████████| 79 kB 4.9 MB/s 
     |████████████████████████████████| 127 kB 58.8 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=bfc90b717f88f

In [ ]:
from kobert.pytorch_kobert import get_pytorch_kobert_model

max_len = 32
batch_size = 32
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
import gluonnlp as nlp

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_val = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
device = torch.device("cuda:0")

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes= len(train_df.digit_3.value_counts()),   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel


model = BERTClassifier(bertmodel,  dr_rate=0.4).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# class weights balancing
class_weights = []

for i in train_df.digit_3.unique():
    weights = (1-len(train_df[train_df.digit_3 == i])/len(train_df))
    class_weights.append(weights)

class_weights_t = torch.tensor(class_weights).cuda()

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss(weight = class_weights_t) # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from tqdm import tqdm, tqdm_notebook

train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} texst acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/3501 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 3.679781913757324 train acc 0.0
epoch 1 batch id 201 loss 3.352102279663086 train acc 0.11302860696517412
epoch 1 batch id 401 loss 2.9201908111572266 train acc 0.19248753117206982
epoch 1 batch id 601 loss 2.0721194744110107 train acc 0.3187915973377704
epoch 1 batch id 801 loss 1.0749125480651855 train acc 0.42388420724094883
epoch 1 batch id 1001 loss 1.058289647102356 train acc 0.5002497502497503
epoch 1 batch id 1201 loss 0.9970738887786865 train acc 0.5625
epoch 1 batch id 1401 loss 0.5093112587928772 train acc 0.6094307637401856
epoch 1 batch id 1601 loss 0.4730498194694519 train acc 0.6480910368519676
epoch 1 batch id 1801 loss 0.5012097358703613 train acc 0.6785813436979455
epoch 1 batch id 2001 loss 0.6532094478607178 train acc 0.7028516991504248
epoch 1 batch id 2201 loss 0.33139556646347046 train acc 0.7236767378464335
epoch 1 batch id 2401 loss 0.2984549403190613 train acc 0.7408241357767597
epoch 1 batch id 2601 loss 0.4265367090702057 train acc 0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 7 batch id 1 loss 0.0596371628344059 train acc 0.96875
epoch 7 batch id 201 loss 0.28113314509391785 train acc 0.9777674129353234
epoch 7 batch id 401 loss 0.12689736485481262 train acc 0.9781795511221946
epoch 7 batch id 601 loss 0.10467101633548737 train acc 0.9777974209650583
epoch 7 batch id 801 loss 0.057292092591524124 train acc 0.9783863920099876
epoch 7 batch id 1001 loss 0.2564527690410614 train acc 0.9781468531468531
epoch 7 batch id 1201 loss 0.17662829160690308 train acc 0.9783513738551207
epoch 7 batch id 1401 loss 0.01748432219028473 train acc 0.9783636688079943
epoch 7 batch id 1601 loss 0.3751482367515564 train acc 0.9784509681449094
epoch 7 batch id 1801 loss 0.1271217316389084 train acc 0.9787791504719601
epoch 7 batch id 2001 loss 0.2089810073375702 train acc 0.9792135182408795
epoch 7 batch id 2201 loss 0.09941522032022476 train acc 0.9796257383007724
epoch 7 batch id 2401 loss 0.1040128841996193 train acc 0.9796308829654311
epoch 7 batch id 2601 loss 0.041476

  0%|          | 0/876 [00:00<?, ?it/s]

epoch 7 texst acc 0.9492009132420092


  0%|          | 0/3501 [00:00<?, ?it/s]

epoch 8 batch id 1 loss 0.004396978300064802 train acc 1.0
epoch 8 batch id 201 loss 0.18189603090286255 train acc 0.9833644278606966
epoch 8 batch id 401 loss 0.08948999643325806 train acc 0.98285536159601


In [ ]:
test_data = test_df[['text']]
test_label = [0 for i in range(len(test_df))]
test_data['label'] = test_label

test_data = [list(test_data.iloc[i]) for i in range(len(test_data))]
print(test_data[:10])
test_data

In [ ]:
data_test = BERTDataset(test_data, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
predictions =[]
with torch.no_grad():
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().cuda()
    segment_ids = segment_ids.long().cuda()
    valid_length= valid_length
    label = label.long().cuda()
    out = model(token_ids, valid_length, segment_ids)
    _, max_indices = torch.max(out, 1)

    predictions.extend(max_indices.squeeze(0).detach().cpu().numpy())


In [ ]:
final = le.inverse_transform(predictions)

In [ ]:
test_df['digit_3'] = final
test_df

In [ ]:
test_df['digit_3'] = test_df['digit_3'].astype(int)

In [ ]:
test_df.info()

In [ ]:
test_df.to_csv('11_Reduction_E10_digit3.csv')

In [ ]:
predictions =[]
with torch.no_grad():
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().cuda()
    segment_ids = segment_ids.long().cuda()
    valid_length= valid_length
    label = label.long().cuda()
    out = model(token_ids, valid_length, segment_ids)
    max_indices = torch.nn.functional.softmax(out, dim=1).detach().cpu().numpy()

    predictions.extend(max_indices)

In [ ]:
x = np.array(predictions)

In [ ]:
np.save('Reduction_E10_digit3', x)